This entire script is used to evaluate retrieval search system using vector search

In [1]:
import json

with open('documents-crypto-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [2]:
from sentence_transformers import SentenceTransformer

In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\sofia\anaconda3\envs\crypto_llm\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sofia\.cache\huggingface\hub\models--sentence-transformers--multi-qa-MiniLM-L6-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

C:\Users\sofia\anaconda3\envs\crypto_llm\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
v = model.encode('what is a blockchain technology?')

In [5]:
len(v)

384

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answer": {"type": "text"},
            "question": {"type": "text"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_answer_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "crypto-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'crypto-questions'})

Creating embedding for answer, question, answer and question

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    question = doc['question']
    answer = doc['answer']
    qt = question + ' ' + answer

    doc['question_vector'] = model.encode(question)
    doc['answer_vector'] = model.encode(answer)
    doc['question_answer_vector'] = model.encode(qt)

  0%|          | 0/46 [00:00<?, ?it/s]

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/46 [00:00<?, ?it/s]

Create search query based on vector

In [20]:
def elastic_search_knn(field, vector):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
    }

    search_query = {
        "knn": knn,
        "_source": ["answer", "question", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [21]:
def question_vector_knn(q):
    question = q['question']
    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q)

In [12]:
import pandas as pd

In [13]:
df_ground_truth = pd.read_csv('ground-truth-data-crypto.csv')

In [14]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [15]:
ground_truth[0]

{'question': 'What is a blockchain?', 'document': 'ccb39dc7', 'total_count': 4}

In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [17]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [30]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        # print(f"Results from search function {results}")
        relevance = [d['id'] == doc_id for d in results]
        # print(f"Relevance for each doc {relevance}")
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [31]:
evaluate(ground_truth, question_vector_knn)

  0%|          | 0/205 [00:00<?, ?it/s]

{'hit_rate': 0.7902439024390244, 'mrr': 0.6491056910569105}

In [32]:
def text_vector_knn(q):
    question = q['question']

    v_q = model.encode(question)

    return elastic_search_knn('answer_vector', v_q)

In [33]:
evaluate(ground_truth, text_vector_knn)

  0%|          | 0/205 [00:00<?, ?it/s]

{'hit_rate': 0.9804878048780488, 'mrr': 0.8544715447154471}

In [35]:
def question_text_vector_knn(q):
    question = q['question']

    v_q = model.encode(question)

    return elastic_search_knn('question_answer_vector', v_q)

evaluate(ground_truth, question_text_vector_knn)

  0%|          | 0/205 [00:00<?, ?it/s]

{'hit_rate': 0.9414634146341463, 'mrr': 0.8426829268292683}

In [40]:
# def elastic_search_knn_combined(vector):
#     search_query = {
#         "size": 5,
#         "query": {
#             "bool": {
#                 "must": [
#                     {
#                         "script_score": {
#                             "script": {
#                                 "source": """
#                                     cosineSimilarity(params.query_vector, 'question_vector') + 
#                                     cosineSimilarity(params.query_vector, 'answer_vector') + 
#                                     cosineSimilarity(params.query_vector, 'question_answer_vector') + 
#                                     1
#                                 """,
#                                 "params": {
#                                     "query_vector": vector
#                                 }
#                             }
#                         }
#                     }
#                 ]
#             }
#         },
#         "_source": ["answer", "question", "id"]
#     }

#     es_results = es_client.search(
#         index=index_name,
#         body=search_query
#     )
    
#     result_docs = []
    
#     for hit in es_results['hits']['hits']:
#         result_docs.append(hit['_source'])

#     return result_docs

In [41]:
# def vector_combined_knn(q):
#     question = q['question']

#     v_q = model.encode(question)

#     return elastic_search_knn_combined(v_q)

# evaluate(ground_truth, vector_combined_knn)